In [1]:
#!pip3 install mlx-lm matplotlib ipywidgets

In [2]:
import json
import os
from typing import Dict, List, Optional, Tuple, Union

import matplotlib.pyplot as plt
import mlx.optimizers as optim
from mlx.utils import tree_flatten
from mlx_lm import generate, load
from mlx_lm.tuner import TrainingArgs, datasets, linear_to_lora_layers, train
from transformers import PreTrainedTokenizer

In [5]:
model_path = "mlx-community/Phi-3-mini-128k-instruct-4bit"
model, tokenizer = load(model_path)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

modeling_phi3.py: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sample_finetune.py: 0.00B [00:00, ?B/s]

In [6]:
prompt = "What is fine-tuning in machine learning?"
messages = [{"role": "user", "content": prompt}]
prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = generate(model, tokenizer, prompt=prompt, verbose=True)

Fine-tuning in machine learning refers to a process where a pre-trained model, which has already been trained on a large dataset, is further trained (or "fine-tuned") on a smaller, more specific dataset to adapt it to a particular task or domain. This is often done to improve performance on a specific problem or to make the model more efficient in handling a particular type of data or task. The pre-trained model, which has already learned a general understanding of the problem space, is adjusted to better suit the specific needs of the task at hand. This is particularly common in deep learning, where large, pre-trained neural networks are used as a starting point for a wide range of applications. The process involves continuing the training of the model on a smaller, task-specific dataset, often with a lower learning rate, to adjust the weights and biases to better perform on the specific task. This method leverages the general knowledge the model has already acquired and adapts it to 

In [8]:
adapter_path = "adapters"
os.makedirs(adapter_path, exist_ok=True)
adapter_config_path = os.path.join(adapter_path, "adapter_config.json")
adapter_file_path = os.path.join(adapter_path, "adapters.safetensors")

In [9]:
lora_config = {
    "num_layers": 8,
    "lora_parameters": {
        "rank": 8,
        "scale": 20.0,
        "dropout": 0.0,
    },
}

In [10]:
with open(adapter_config_path, "w") as f:
    json.dump(lora_config, f, indent=4)